# Day 10
https://adventofcode.com/2016/day/10

In [1]:
import aocd
data = aocd.get_data(year=2016, day=10)

In [2]:
from dataclasses import dataclass
from typing import Dict, List
import re

In [3]:
re_value = re.compile(r'value (\d+) goes to (\w+) (\d+)')
re_robot = re.compile(r'bot (\d+) gives low to (\w+) (\d+) and high to (\w+) (\d+)')

In [4]:
@dataclass(frozen=True)
class Target():
    genre: str
    number: int

In [5]:
@dataclass
class Robot():
    holding: List[int]
    give_low: Target
    give_high: Target
    
    def __init__(self, low_genre, low_number, high_genre, high_number):
        self.holding = []
        self.give_low = Target(low_genre, int(low_number))
        self.give_high = Target(high_genre, int(high_number))

    def add(self, value, callback):
        self.holding.append(value)
        if len(self.holding) == 2:
            callback(self.give_low, min(self.holding))
            callback(self.give_high, max(self.holding))

In [6]:
@dataclass
class Environment:
    robots: Dict[int, Robot]
    outputs: Dict[int, int]
    
    def __init__(self, instructions):
        self.robots = {}
        self.outputs = {}
        
        for robot_definition in re_robot.findall(instructions):
            bot, args = robot_definition[0], robot_definition[1:]
            self.robots[int(bot)] = Robot(*args)
        
        for (initial_value, to_genre, to_number) in re_value.findall(instructions):
            target = Target(to_genre, int(to_number))
            self.deliver(target, int(initial_value))
    
    def deliver(self, target, value):
        if target.genre == 'bot':
            self.robots[target.number].add(value, self.deliver)
        elif target.genre == 'output':
            self.outputs[target.number] = value
    
    def find_robot_holding(self, search):
        inventory = set(search)
        for number, bot in self.robots.items():
            if set(bot.holding) == inventory:
                return number

In [7]:
env = Environment(data)
p1 = env.find_robot_holding((61, 17))
print('Part 1: {}'.format(p1))
p2 = env.outputs[0] * env.outputs[1] * env.outputs[2]
print('Part 2: {}'.format(p2))

Part 1: 93
Part 2: 47101
